In [1]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [2]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [3]:
from IPython.core.display import HTML
HTML(r"""
<style>
    * {
        font-family: Monaco;
    }
</style>
""")

In [4]:
import os,sys,warnings,re,math,gc,time
warnings.filterwarnings("ignore")
os.cpu_count()

12

In [5]:
# os.environ["POLARS_MAX_THREADS"] = "20"
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco",size=15)))
pio.templates.default = "mod+plotly_dark"
from sklearn.metrics import auc,roc_auc_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
%xmode Minimal

Exception reporting mode: Minimal


In [6]:
pl.Config.set_fmt_str_lengths(100)
# pl.Config.set_streaming_chunk_size(10000)
pl.Config.set_thousands_separator(",")
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(40)

polars.config.Config

In [7]:
path_to_train = Path("Downloads/credit_risk/train/")
path_to_test = Path("Downloads/credit_risk/test/")
path_to_features = Path("Downloads/feature_definitions.csv")

In [8]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [9]:
all_train_files = glob("Downloads/credit_risk/train/*.parquet")
all_test_files = glob("Downloads/credit_risk/test/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [10]:
display(train_files_df.filter(pl.col("filename").str.contains("base")))
_ind = train_files_df.filter(pl.col("filename").str.contains("base")).select("index").item()

index,path,filename
i64,str,str
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""


In [11]:
train_base = (
    pl.scan_parquet(all_train_files[_ind])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains("target"),cs.contains("target"))
)
display(train_base.fetch().head())
print(train_base.collect(streaming=True).shape)

case_id_base,date,month,week,weekday,target
u32,date,i8,i8,i8,u8
0,2019-01-03,1,1,4,0
1,2019-01-03,1,1,4,0
2,2019-01-04,1,1,5,0
3,2019-01-03,1,1,4,0
4,2019-01-04,1,1,5,1


(1526659, 6)


In [12]:
display(test_files_df.filter(pl.col("filename").str.contains("base")))
_ind = test_files_df.filter(pl.col("filename").str.contains("base")).select("index").item()

index,path,filename
i64,str,str
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [13]:
test_base = (
    pl.scan_parquet(all_test_files[_ind])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday"),
        pl.lit(0).cast(pl.UInt8).alias("target")
    )
)
display(test_base.fetch().head())
print(test_base.collect().shape)

case_id_base,date,month,week,weekday,target
u32,date,i8,i8,i8,u8
"57,543",2020-10-06,10,41,2,0
"57,549",2020-10-06,10,41,2,0
"57,551",2020-10-06,10,41,2,0
"57,552",2020-10-07,10,41,3,0
"57,569",2020-10-06,10,41,2,0


(10, 6)


In [14]:
all_files_df = (
    pl.concat(
        [
            train_files_df,
            test_files_df
        ]
    )
    .sort(by="filename")
)
all_files_df.head()

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [15]:
total_base = (
    pl.concat(
        [
            train_base,
            test_base
        ]
    )
)
total_base.fetch().head()

case_id_base,date,month,week,weekday,target
u32,date,i8,i8,i8,u8
0,2019-01-03,1,1,4,0
1,2019-01-03,1,1,4,0
2,2019-01-04,1,1,5,0
3,2019-01-03,1,1,4,0
4,2019-01-04,1,1,5,1


In [16]:
def read_and_cast(filter_string:str):
    _files_list = all_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list()
    return (
        pl.concat(
            [
                pl.scan_parquet(_) for _ in _files_list
            ],
            how="vertical_relaxed"
        )
        .select(
            cs.by_name("case_id").cast(pl.UInt32),
            cs.contains("num_group").cast(pl.UInt16),
            cs.ends_with("D").cast(pl.Date),
            cs.ends_with("L","T","M").cast(pl.String).cast(pl.Categorical),
            cs.ends_with("P","A").cast(pl.Float32)
        )
        .sort(by="case_id")
    )



def grouping(df:pl.LazyFrame):
    return (
        df
        .group_by("case_id")
        .agg(
            (cs.ends_with("P","A","D") | cs.contains("num_group")).max().suffix("_max"),
            (cs.ends_with("P","A","D") | cs.contains("num_group")).min().suffix("_min"),
            (cs.ends_with("P","A","D") | cs.contains("num_group")).mean().suffix("_mean"),
            (cs.ends_with("P","A","D") | cs.contains("num_group")).std().suffix("_std"),
            cs.ends_with("L","T","M").mode().drop_nulls().first().suffix("_first_mode"),
            cs.ends_with("L","T","M").mode().drop_nulls().last().suffix("_last_mode"),
            cs.all().first().suffix("_first"),
            cs.all().last().suffix("_last")
        )
        .select(
            cs.by_name("case_id"),
            cs.contains("num_group"),
            ~(cs.by_name("case_id") | cs.contains("num_group"))
        )
    )

## Past Application

In [17]:
total_past_shallow = read_and_cast("applprev_1")
total_past_shallow.fetch().head()

case_id,num_group1,approvaldate_319D,creationdate_885D,dateactivated_425D,dtlastpmt_581D,dtlastpmtallstes_3545839D,employedfrom_700D,firstnonzeroinstldate_307D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,credacc_status_367L,credacc_transactions_402L,credtype_587L,district_544M,education_1138M,familystate_726L,inittransactioncode_279L,isbidproduct_390L,isdebitcard_527L,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,status_219L,tenor_203L,actualdpd_943P,annuity_853A,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,revolvingaccount_394A
u32,u16,date,date,date,date,date,date,date,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2,0,null,2013-04-03,null,null,null,2010-02-15,2013-05-04,null,"""a55475b1""","""0.0""",null,null,"""CAL""","""P136_108_173""","""P97_36_170""","""SINGLE""","""CASH""","""false""",null,"""24.0""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""","""24.0""",0.00,640.20,null,0.00,null,null,"10,000.00",null,0.00,"8,200.00",null,null,null
2,1,null,2013-04-03,null,null,null,2010-02-15,2013-05-04,null,"""a55475b1""","""0.0""",null,null,"""CAL""","""P136_108_173""","""P97_36_170""","""SINGLE""","""CASH""","""false""",null,"""12.0""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""","""12.0""",0.00,"1,682.40",null,0.00,null,null,"16,000.00",null,0.00,"8,200.00",null,null,null
3,0,null,2019-01-07,null,null,null,2018-05-15,2019-02-07,null,"""P94_109_143""",null,null,null,"""CAL""","""P131_33_167""","""P97_36_170""","""MARRIED""","""CASH""","""false""",null,"""12.0""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""12.0""",0.00,"6,140.00",null,0.00,null,null,"59,999.80",null,0.00,"11,000.00",null,null,null
4,0,null,2019-01-08,null,null,null,null,2019-02-08,null,"""P24_27_36""",null,null,null,"""CAL""","""P194_82_174""","""a55475b1""",null,"""CASH""","""false""",null,"""24.0""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""","""24.0""",0.00,"2,556.60",null,0.00,null,null,"40,000.00",null,0.00,"16,000.00",null,null,null
5,0,null,2019-01-16,null,null,null,null,null,null,"""P85_114_140""",null,null,null,null,"""P54_133_26""","""a55475b1""",null,null,"""false""",null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""",null,0.00,null,null,null,null,null,null,null,null,"62,000.00",null,null,null


In [18]:
total_past_depth = read_and_cast("applprev_2")
total_past_depth.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L
u32,u16,u16,cat,cat,cat
2,1,1,null,"""EMPLOYMENT_PHONE""",null
2,0,1,null,"""EMPLOYMENT_PHONE""",null
2,0,0,null,"""PRIMARY_MOBILE""",null
2,1,0,null,"""PRIMARY_MOBILE""",null
3,0,1,null,"""PRIMARY_MOBILE""",null


In [19]:
total_static_base = read_and_cast("static_0")
total_static_base.fetch().head()

case_id,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,dtlastpmtallstes_4499206D,firstclxcampaign_1125D,firstdatedue_489D,lastactivateddate_801D,lastapplicationdate_877D,lastapprdate_640D,lastdelinqdate_224D,lastrejectdate_50D,lastrepayingdate_696D,maxdpdinstldate_3546855D,payvacationpostpone_4187118D,validfrom_1069D,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,…,disbursedcredamount_1113A,downpmt_116A,inittransactionamount_650A,lastapprcredamount_781A,lastotherinc_902A,lastotherlnsexpense_631A,lastrejectcredamount_222A,maininc_215A,maxannuity_159A,maxannuity_4075009A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A
u32,date,date,date,date,date,date,date,date,date,date,date,date,date,date,date,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""",null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""",…,"30,000.00",0.00,null,null,null,null,null,null,0.00,null,null,null,null,0.00,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,0.00,0.00,null,null,null,null,0.00,0.00,null
1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""3.0""","""0.0""","""0.0""","""0.0""","""0.0""",…,"19,999.80",0.00,null,null,null,null,null,null,0.00,null,null,null,null,0.00,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,0.00,0.00,null,null,null,null,0.00,0.00,null
2,null,null,null,null,null,null,null,2013-04-03,null,null,2013-04-03,null,null,null,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""",null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""",…,"78,000.00",0.00,null,null,null,null,"10,000.00",null,0.00,null,null,null,null,0.00,null,null,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,0.00,0.00,null,null,null,null,0.00,0.00,null
3,null,null,null,null,null,null,null,2019-01-07,null,null,2019-01-07,null,null,null,null,"""0.0""","""1.0""","""0.0""","""2.0""","""0.0""","""1.0""",null,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""1.0""",null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""",…,"40,000.00",0.00,null,null,null,null,"59,999.80",null,0.00,null,null,null,null,0.00,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,0.00,0.00,null,null,null,null,0.00,0.00,null
4,null,null,null,null,null,null,null,2019-01-08,null,null,null,null,null,null,null,"""0.0""","""1.0""","""0.0""","""0.0""","""0.0""","""1.0""",null,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""

In [20]:
total_person_shallow = read_and_cast("person_1")
total_person_shallow.fetch().head()

case_id,num_group1,birth_259D,birthdate_87D,empl_employedfrom_271D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedtotal_800L,empl_industry_691L,empladdr_district_926M,empladdr_zipcode_114M,familystate_447L,gender_992L,housetype_905L,housingtype_772L,incometype_1044T,isreference_387L,language1_981M,maritalst_703L,personindex_1023L,persontype_1072L,persontype_792L,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L,mainoccupationinc_384A
u32,u16,date,date,date,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f32
0,0,1986-07-01,null,2017-09-15,null,"""P88_18_84""","""false""","""false""","""P167_100_165""","""P97_36_170""","""MORE_FIVE""","""OTHER""","""P142_57_166""","""P167_100_165""","""MARRIED""",null,null,null,"""SALARIED_GOVT""",null,"""P10_39_147""",null,"""0.0""","""1.0""","""1.0""","""P88_18_84""","""P167_100_165""",null,null,null,"""CL""",null,"""true""","""F""","""PRIMARY_MOBILE""","10,800.00"
0,1,null,null,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,"""1.0""","""1.0""","""4.0""","""a55475b1""","""a55475b1""","""SPOUSE""",null,"""false""","""EM""",null,null,null,"""PHONE""",null
0,2,null,null,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,"""2.0""","""4.0""","""5.0""","""a55475b1""","""a55475b1""","""COLLEAGUE""","""SPOUSE""","""false""","""PE""",null,null,null,"""PHONE""",null
0,3,null,null,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,null,"""5.0""",null,"""a55475b1""","""a55475b1""",null,"""COLLEAGUE""",null,"""PE""",null,null,null,"""PHONE""",null
1,0,1957-08-01,null,2008-10-29,null,"""P103_93_94""","""false""","""false""","""P176_37_166""","""P97_36_170""","""MORE_FIVE""","""OTHER""","""P49_46_174""","""P160_59_140""","""DIVORCED""",null,null,null,"""SALARIED_GOVT""",null,"""P10_39_147""",null,"""0.0""","""1.0""","""1.0""","""P103_93_94""","""P176_37_166""",null,null,null,"""CL""",null,"""true""","""M""","""PRIMARY_MOBILE""","10,000.00"


In [21]:
total_person_depth = read_and_cast("person_2")
total_person_depth.fetch().head()

case_id,num_group1,num_group2,empls_employedfrom_796D,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employer_name_740M,relatedpersons_role_762T
u32,u16,u16,date,cat,cat,cat,cat,cat,cat,cat
5,0,0,null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null
6,0,0,null,"""P55_110_32""","""CONTACT""","""P10_68_40""","""P38_92_157""","""P164_110_33""","""a55475b1""",null
6,0,1,null,"""P55_110_32""","""PERMANENT""","""P10_68_40""","""a55475b1""","""a55475b1""","""a55475b1""",null
6,1,0,null,"""P204_92_178""","""CONTACT""","""P65_136_169""","""P38_92_157""","""P164_110_33""","""a55475b1""","""OTHER_RELATIVE"""
6,1,1,null,"""P191_109_75""","""CONTACT""","""P10_68_40""","""P7_147_157""","""a55475b1""","""a55475b1""","""OTHER_RELATIVE"""


In [22]:
total_other_shallow = read_and_cast("other_1")
total_other_shallow.fetch().head()

case_id,num_group1,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A
u32,u16,f32,f32,f32,f32,f32
"43,801",0,"12,466.60","12,291.20",914.20,0.00,304.80
"43,991",0,"3,333.40","3,273.40",0.00,0.00,0.00
"44,001",0,"10,000.00","10,000.00",0.00,0.00,0.00
"44,053",0,0.00,0.00,"2,586.40",0.00,88.80
"44,130",0,63.80,60.80,0.00,0.00,0.00


In [23]:
total_deposit_shallow = read_and_cast("deposit_1")
total_deposit_shallow.fetch().head()

case_id,num_group1,contractenddate_991D,openingdate_313D,amount_416A
u32,u16,date,date,f32
225,0,null,2016-08-16,0.00
331,0,2018-03-18,2015-03-19,260.37
358,0,null,2014-09-02,0.00
390,0,2017-07-22,2014-07-23,"211,748.53"
390,2,null,2016-06-08,223.68


In [24]:
total_debitcard_shallow = read_and_cast("debitcard")
total_debitcard_shallow.fetch().head()

case_id,num_group1,openingdate_857D,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A
u32,u16,date,f32,f32,f32
225,0,2016-08-16,null,null,null
331,0,2015-03-19,null,null,null
358,0,2014-09-02,null,null,null
390,0,2014-07-23,null,null,null
390,2,2016-06-08,null,null,null


In [25]:
total_credit_internal_shallow = read_and_cast("bureau_a_1")
total_credit_internal_shallow.fetch().head()

case_id,num_group1,dateofcredend_289D,dateofcredend_353D,dateofcredstart_181D,dateofcredstart_739D,dateofrealrepmt_138D,lastupdate_1112D,lastupdate_388D,numberofoverdueinstlmaxdat_148D,numberofoverdueinstlmaxdat_641D,overdueamountmax2date_1002D,overdueamountmax2date_1142D,refreshdate_3813885D,annualeffectiverate_199L,annualeffectiverate_63L,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,contractsum_5085717L,description_351M,dpdmaxdatemonth_442T,dpdmaxdatemonth_89T,dpdmaxdateyear_596T,dpdmaxdateyear_896T,financialinstitution_382M,financialinstitution_591M,interestrate_508L,nominalrate_281L,nominalrate_498L,numberofcontrsvalue_258L,numberofcontrsvalue_358L,numberofinstls_229L,numberofinstls_320L,numberofoutstandinstls_520L,numberofoutstandinstls_59L,…,overdueamountmaxdatemonth_365T,overdueamountmaxdateyear_2T,overdueamountmaxdateyear_994T,periodicityofpmts_1102L,periodicityofpmts_837L,prolongationcount_1120L,prolongationcount_599L,purposeofcred_426M,purposeofcred_874M,subjectrole_182M,subjectrole_93M,credlmt_230A,credlmt_935A,debtoutstand_525A,debtoverdue_47A,dpdmax_139P,dpdmax_757P,instlamount_768A,instlamount_852A,monthlyinstlamount_332A,monthlyinstlamount_674A,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,overdueamountmax2_398A,overdueamountmax_155A,overdueamountmax_35A,residualamount_488A,residualamount_856A,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A
u32,u16,date,date,date,date,date,date,date,date,date,date,date,date,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,…,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
388,0,2023-06-20,null,null,2018-06-20,null,2019-01-24,null,null,null,null,null,null,null,null,"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""",null,"""a55475b1""",null,"""7.0""","""2018.0""",null,"""a55475b1""","""55b002a9""",null,null,null,"""2.0""","""1.0""",null,"""60.0""",null,"""54.0""",…,"""7.0""","""2018.0""",null,null,"""30.0""",null,null,"""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf""",null,null,"374,419.50",0.00,0.00,null,null,null,"7,811.45",null,null,"260,093.70",null,0.00,0.00,null,0.00,null,null,null,null,"268,897.62",0.00,0.00,"374,419.50",0.00
388,1,2020-08-06,null,null,2018-08-06,null,2019-01-11,null,null,null,null,null,null,null,null,"""ea6782cc""","""a55475b1""","""7241344e""","""a55475b1""",null,"""a55475b1""",null,"""8.0""","""2018.0""",null,"""a55475b1""","""P204_66_73""",null,null,null,null,null,null,null,null,null,…,"""8.0""","""2018.0""",null,null,null,null,null,"""60c73645""","""a55475b1""","""a55475b1""","""a55475b1""",null,"135,806.00",null,null,0.00,null,"8,742.80",null,"8,742.80",null,null,null,null,0.00,0.00,null,0.00,null,null,"114,325.80",null,null,null,null,null,null
388,2,null,null,null,null,null,null,null,null,null,null,null,2019-01-28,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
388,3,null,null,null,null,null,null,null,null,null,null,null,2019-01-28,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
388,4,null,n

In [26]:
total_credit_external_shallow = read_and_cast("bureau_b_1")
total_credit_external_shallow.fetch().head()

case_id,num_group1,contractdate_551D,contractmaturitydate_151D,lastupdate_260D,classificationofcontr_1114M,contractst_516M,contracttype_653M,credor_3940957M,credquantity_1099L,credquantity_984L,dpdmaxdatemonth_804T,dpdmaxdateyear_742T,interesteffectiverate_369L,interestrateyearly_538L,numberofinstls_810L,overdueamountmaxdatemonth_494T,overdueamountmaxdateyear_432T,periodicityofpmts_997L,periodicityofpmts_997M,pmtmethod_731M,pmtnumpending_403L,purposeofcred_722M,subjectrole_326M,subjectrole_43M,amount_1115A,credlmt_1052A,credlmt_228A,credlmt_3940954A,debtpastduevalue_732A,debtvalue_227A,dpd_550P,dpd_733P,dpdmax_851P,installmentamount_644A,installmentamount_833A,instlamount_892A,maxdebtpduevalodued_3940955A,overdueamountmax_950A,pmtdaysoverdue_1135P,residualamount_1093A,residualamount_127A,residualamount_3940956A,totalamount_503A,totalamount_881A
u32,u16,date,date,date,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
467,0,2011-06-15,2031-06-13,2019-01-20,"""ea6782cc""","""7241344e""","""724be82a""","""P164_34_168""","""2.0""","""1.0""",null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,"""96a8fdfe""","""fa4f56f1""","""ab3c25cf""",null,"3,000,000.00","10,000.00","3,000,000.00",null,null,0.00,0.00,null,0.00,0.00,null,null,null,null,0.00,0.00,null,"3,000,000.00","10,000.00"
467,1,2019-01-04,2021-08-04,2019-01-20,"""ea6782cc""","""7241344e""","""724be82a""","""P164_34_168""","""1.0""","""2.0""",null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,"""96a8fdfe""","""ab3c25cf""","""ab3c25cf""",null,null,null,"130,365.00",null,null,0.00,0.00,null,0.00,"26,571.97",null,null,null,null,null,null,null,"78,000.00","960,000.00"
467,2,2016-10-25,2019-10-25,2019-01-10,"""ea6782cc""","""7241344e""","""4257cbed""","""c5a72b57""",null,null,"""11.0""","""2016.0""",null,null,"""36.0""","""11.0""","""2016.0""",null,"""a0b598e4""","""e914c86c""","""10.0""","""96a8fdfe""","""a55475b1""","""a55475b1""","78,000.00",null,null,null,0.00,"26,571.97",null,null,0.00,null,null,"2,898.76",0.00,0.00,0.00,null,null,null,null,null
"1,445",0,2015-01-30,2021-01-30,2019-01-19,"""ea6782cc""","""7241344e""","""1c9c5356""","""b619fa46""","""2.0""","""5.0""","""1.0""","""2018.0""",null,null,null,"""2.0""","""2018.0""",null,"""a55475b1""","""a55475b1""",null,"""60c73645""","""daf49a8a""","""ab3c25cf""",null,"400,000.00","100,000.00","74,000.00",0.00,null,0.00,0.00,"200,418.00",0.00,0.00,null,0.40,1.40,0.00,0.00,0.00,"73,044.18","400,000.00","100,000.00"
"1,445",1,2014-09-12,2021-09-12,2019-01-13,"""01f63ac8""","""7241344e""","""724be82a""","""74bd67a8""","""3.0""","""17.0""",null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,"""96a8fdfe""","""ab3c25cf""","""ab3c25cf""",null,null,null,"400,000.00",null,null,0.00,0.00,null,0.00,"209,617.77",null,null,null,null,null,null,null,"396,800.62","184,587.80"


In [27]:
total_credit_external_depth = read_and_cast("bureau_b_2")
total_credit_external_depth.fetch().head()

case_id,num_group1,num_group2,pmts_date_1107D,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A
u32,u16,u16,date,f32,f32
467,2,19,2018-05-15,0.00,0.00
467,2,25,2018-11-15,0.00,0.00
467,2,18,2018-04-15,0.00,0.00
467,2,0,2016-10-15,0.00,0.00
467,2,6,2017-04-15,0.00,0.00


In [28]:
total_registry_a = read_and_cast("registry_a")
total_registry_a.fetch().head()

case_id,num_group1,recorddate_4527225D,name_4527232M,amount_4527230A
u32,u16,date,cat,f32
"28,631",2,2019-09-13,"""f980a1ea""","1,946.00"
"28,631",3,2019-09-13,"""f980a1ea""",711.00
"28,631",0,2019-09-13,"""f980a1ea""","3,616.40"
"28,631",1,2019-09-13,"""f980a1ea""","2,600.00"
"28,632",5,2019-09-13,"""5f9b74f5""","2,564.00"


In [29]:
total_registry_b = read_and_cast("registry_b")
total_registry_b.fetch().head()

case_id,num_group1,deductiondate_4917603D,name_4917606M,amount_4917619A
u32,u16,date,cat,f32
"49,435",7,2019-10-16,"""6b730375""","6,885.00"
"49,435",1,2019-10-16,"""6b730375""","6,885.00"
"49,435",8,2019-10-16,"""6b730375""","6,885.00"
"49,435",3,2019-10-16,"""6b730375""","6,885.00"
"49,435",4,2019-10-16,"""6b730375""","6,885.00"


In [30]:
total_registry_c = read_and_cast("registry_c")
total_registry_c.fetch().head()

case_id,num_group1,processingdate_168D,employername_160M,pmtamount_36A
u32,u16,date,cat,f32
357,5,2018-08-08,"""c91b12ff""","1,100.00"
357,1,2018-11-28,"""c91b12ff""","1,200.00"
357,4,2018-09-10,"""c91b12ff""","1,200.00"
357,0,2019-01-04,"""c91b12ff""","1,200.00"
357,3,2018-10-08,"""c91b12ff""",628.60


In [31]:
try:
    for _ in globals().keys():
        if "total_" in _:
            print(_)
except RuntimeError:
    for _ in globals().keys():
        if "total_" in _:
            print(_)

total_base
total_past_shallow
total_past_depth
total_static_base
total_person_shallow
total_person_depth
total_other_shallow
total_deposit_shallow
total_debitcard_shallow
total_credit_internal_shallow
total_credit_external_shallow
total_credit_external_depth
total_registry_a
total_registry_b
total_registry_c


In [32]:
total_df = (
    total_base
    .join(
        total_past_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_past_depth.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_static_base.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_depth.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_other_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_deposit_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_debitcard_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_internal_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_depth.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_a.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_b.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_c.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .select(
        (cs.ends_with("D") - pl.col("date"))
    )
)
total_df.collect(streaming=True)